### 保存和加载模型代码
+ 为什么要保存模型代码：可以从任意中断恢复，可以共享模型代码
+ keras构造的模型保存的格式是.h5
+ tensorflow构造的模型保存的格式是.pb

本文来源 https://tensorflow.google.cn/tutorials/keras/save_and_load#%E9%80%89%E9%A1%B9

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'                  # 必须放在首行，不然会先导入tensorflow，这一行的命令就无效了！

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
print(tf.version.VERSION)
print (device_lib.list_local_devices())

#### 1. 获取示例数据集

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### 2. 构造模型

In [ ]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = create_model()
# 显示模型的结构
model.summary()

#### 3.1 在训练期间保存模型（以checkpoint形式保存）
tf.keras.callbacks.ModelCheckpoint允许在训练的过程中和结束时回调保存的模型。  


In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# 使用新的回调训练模型
model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images,test_labels),
         callbacks=[cp_callback])                     # 通过回调训练

# 这可能会生成与保存程序优化状态相关的警告，可以忽略
# 模型保存产生的文件说明：
# --checkpoint： 说明这是一个checkpoint
# --cp.ckpt.data-00000-of-00001：包含模型全中的分片。如果只在一个机器上训练，只产生这一个
# --cp.ckpt.index：索引文件，指示哪些权重存储在哪个分片中

In [ ]:
# 创建一个新的未经训练的模型。仅恢复模型的权重
model = create_model()

# 评估模型
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained Model, accuracy: {:5.2f}%".format(100*acc) )

In [ ]:
# 加载权重
model.load_weights(checkpoint_path)

# 重新评估模型
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored Model, accuracy: {:5.2f}%".format(100*acc) )

#### 3.2 checkpoint回调选项：每隔5个epoch保存checkpoint

In [ ]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 创建一个新的模型实例
model = create_model()

# 使用 `checkpoint_path` 格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回调*训练*模型
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

In [ ]:
# 选择最新的 checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

# 创建一个新的模型实例
model = create_model()

# 加载以前保存的权重
model.load_weights(latest)

# 重新评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

#### 3.3 手动保存权重

In [ ]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建模型实例
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

#### 3.4 保存整个模型
&emsp;&emsp;模型和优化器可以保存到包含其状态（权重和变量）和模型参数的文件中。这可以让您导出模型，以便在不访问原始 python 代码的情况下使用它。而且您可以通过恢复优化器状态的方式，从中断的位置恢复训练。  
&emsp;&emsp;保存完整模型会非常有用——您可以在 TensorFlow.js (HDF5, Saved Model) 加载他们，然后在 web 浏览器中训练和运行它们，或者使用 TensorFlow Lite 将它们转换为在移动设备上运行(HDF5, Saved Model)  
#### 这项技术可以保存：
+ 权重
+ 模型结构
+ 优化器配置

In [ ]:
# 创建一个新的模型实例
model = create_model()

# 训练模型
model.fit(train_images, train_labels, epochs=5)

# 将整个模型保存为HDF5文件
model.save('my_model.h5')

In [ ]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = keras.models.load_model('my_model.h5')

# 显示网络结构
new_model.summary()

loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))